In [188]:
import sys
sys.path.append('../../process')
import psml
import imp
import numpy as np
import imp
import sys
import copy

In [189]:
trace_dir = 'traces/'

In [190]:
imp.reload(psml)
#Load all for loop traces into one 3d array.
trc_paths = !ls traces/*act*
trcs = psml.paths_to_tracearr(trc_paths, 'act')
trcs.shape

(1, 2639, 5)

In [191]:
#Read events
t0 = trcs[0]
#% 8 for Lower 3 bits = id. id = 1 and 3 are read events
rd_events = ((t0[:,0] % 8) == 1) | ((t0[:,0] % 8) == 3)
rd_events.sum()

1839

In [192]:
#Write events
#% 8 for Lower 3 bits = id. id = 2 and 4 are write events
wr_events = ((t0[:,0] % 8) == 2) | ((t0[:,0] % 8) == 4)
wr_events.sum()

800

In [193]:
#Do they sum to entire trace?
rd_events.sum() + wr_events.sum() == trcs.shape[1]

True

In [194]:
#Trying to stay consistent with sv interpretation.
#No ir here, but is in sv at position 7. Leaving it open here.

#Reg reads
reg_rds = trcs[:,np.squeeze((trcs[:,:,0] % 8) == 3), :]

#Reg addresses
#Pc is 2 bytes. Addresses from simulator don't account for 2nd byte of pc.
reg_mask = np.unique(np.squeeze(reg_rds)[:,1]) + 1

#If Pc was one of the reads, add its 2nd byte
if 1 in reg_mask:
    reg_mask = np.append([0], reg_mask) 

#Start mem component of sv at index 8, skipping over 7.
#Mem_rds
mem_rds = trcs[:,np.squeeze((trcs[:,:,0] % 8) == 1), :]
#Mem addrs can span 2 bytes
mem_addr = np.squeeze(mem_rds)[:,1]+ 256*np.squeeze(mem_rds)[:,2]

#Shift so they start at 8
mem_mask = np.unique(np.array(mem_addr, dtype=np.uint32))+8

#reg_mask
read_mask = np.union1d(reg_mask, mem_mask)
read_mask

array([    0,     1,     2,     3,     4,     5,     6,    16,    17,
          18,    19,    24,   518,   519,   520,   521,   522,   523,
         524,   525,   526,   527,   528,   529,   530,   531,   532,
         533,   534,   535,   536,   537,   538,   539,   540,   541,
         542,   543,   544,   545,   546,   547,   548,   549,   550,
         551,   552,   553,   554,   555,   556,  1288,  1289,  1290,
        1291,  1292,  1293,  1294,  1295,  1296,  1297,  1298,  1299,
        1300,  1301,  1302,  1303,  1304,  1305,  1306,  1307,  1308,
        1309,  1310,  1311,  1312,  1313,  1314,  1315,  1316,  1317,
        1318,  1319,  1327,  1328,  1329,  1330,  1331,  1332,  1333,
        1334,  1335,  1336,  1337,  1338,  1339,  1340,  1341,  1342,
        1353,  1354,  1355,  1356,  1362,  1363,  1364,  1365,  1366,
        1367,  1368,  1369,  1370,  1378,  1379,  1380,  1381,  1382,
        1383,  1384,  1385,  1386,  1387,  1388,  1389,  1390,  1391,
        1392,  1393,

In [195]:
#Write mask
reg_writes = trcs[:,np.squeeze((trcs[:,:,0] % 8) == 4), :]
#Bump over for 2byte pc
reg_mask = np.unique(np.squeeze(reg_writes)[:,1]) + 1
#Grab 2nd byte of pc
if 1 in reg_mask:
    reg_mask = np.append([0], reg_mask) 

#Mem writes
mem_writes = trcs[:,np.squeeze((trcs[:,:,0] % 8) == 2), :]
#Mem addrs can span 2 bytes
mem_addr = np.squeeze(mem_writes)[:,1]+ 256*np.squeeze(mem_writes)[:,2]

#Shift so they start at 8
mem_mask = np.unique(np.array(mem_addr, dtype=np.uint32))+8

#Entire write mask
write_mask = np.union1d(reg_mask, mem_mask)
write_mask

array([  0,   1,   2,   3,   4,   5,   6,   8,   9,  16,  17,  18,  19,
        24, 518, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530,
       531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543,
       544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556])

In [196]:
sv_trc = np.array([psml.path_to_svarr('traces/hi.img.sv.trc')])
psml.mask_svarr(sv_trc, write_mask).shape


(1, 273, 52)

In [197]:
#Append zero mem state.
img = psml.img_to_sv('images/hi.img')

In [198]:
#They do not agree because the 0th row of the sv is AFTER applying the first instruction.
#0 when they agree.
(~(img == sv_trc[0,0,:])).sum()


2

In [199]:
#Actions induced by first instruction to state.
#There are 9 of them by inspection 
#Actions:
'''
9 ir=0x00, ACTCNT=9 (inst=11, BRK, interrupt mode=6, impl: Implied i)
REG RD: sr 00
REG WR: sr 10
REG RD: sr 10
REG WR: sr 10
REG RD: sr 10
REG WR: sr 14
MEM RD: fffd 05
MEM RD: fffc 00
REG WR: pc 0500
'''
#Encoded: 
'''
[ 11,   5,   0,   0,   0],
[ 12,   5,   0,  16,   0],
[ 11,   5,   0,  16,   0],
[ 12,   5,   0,  16,   0],
[ 11,   5,   0,  16,   0],
[ 12,   5,   0,  20,   0],
[  9, 253, 255,   5,   0],
[  9, 252, 255,   0,   0],
[ 20,   0,   0,   0,   5]
'''

'\n[ 11,   5,   0,   0,   0],\n[ 12,   5,   0,  16,   0],\n[ 11,   5,   0,  16,   0],\n[ 12,   5,   0,  16,   0],\n[ 11,   5,   0,  16,   0],\n[ 12,   5,   0,  20,   0],\n[  9, 253, 255,   5,   0],\n[  9, 252, 255,   0,   0],\n[ 20,   0,   0,   0,   5]\n'

In [200]:
#Apply the actions corresponding to the first instruction.
to_apply = trcs[0,:9,:]
to_apply

array([[ 11,   5,   0,   0,   0],
       [ 12,   5,   0,  16,   0],
       [ 11,   5,   0,  16,   0],
       [ 12,   5,   0,  16,   0],
       [ 11,   5,   0,  16,   0],
       [ 12,   5,   0,  20,   0],
       [  9, 253, 255,   5,   0],
       [  9, 252, 255,   0,   0],
       [ 20,   0,   0,   0,   5]], dtype=uint8)

In [201]:
def write_mask_act(act_trc):
    #Write mask
    reg_writes = act_trc[((act_trc % 8)[:,0]) == 4, :]
    #Bump over for 2byte pc
    reg_mask = np.unique(np.squeeze(reg_writes)[:,1]) + 1
    #Grab 2nd byte of pc
    if 1 in reg_mask:
        reg_mask = np.append([0], reg_mask) 

    #Mem writes
    mem_writes = act_trc[((act_trc % 8)[:,0]) == 2, :]
    #Mem addrs can span 2 bytes
    mem_addr = np.squeeze(mem_writes)[:,1]+ 256*np.squeeze(mem_writes)[:,2]

    #Shift so they start at 8
    mem_mask = np.unique(np.array(mem_addr, dtype=np.uint32))+8

    #Entire write mask
    return np.union1d(reg_mask, mem_mask)

In [202]:
#Wrote to pc and 6th register (sr)
write_mask_act(trcs[0,:9,:])

array([0, 1, 6])

In [203]:
def apply_write_actions(state, acts):
    acts = strip_reads_act(acts)
    #If the action was a read, return
    if acts.shape[0] == 0:
        return state
    two_byte_act = two_byte_action_mask(acts)
    #Create mask of 2 byte actions
    #This will probably become a bottleneck
#    assert len(two_byte_act) == len(acts)
    state_cpy = copy.deepcopy(state)
    for a,t in zip(acts, two_byte_act):
        #Does the header indicate a mem or reg write?
        if (a[0] % 8) == 4:
            #Is this a 2 byte action? 
            if t:
                #There is only one 2 byte reg, the pc.
                #Think this is correct order.
                state_cpy[0] = a[3]
                state_cpy[1] = a[4]
            else:
                #Increment to account for 2 byte pc.
                state_cpy[a[1]+1] = a[3]
        #Memory write.
        elif (a[0] % 8) == 2:
            #Calculate sv offset. Add 8 for regs.
            sv_offset = a[1] + 256*a[2] + 8
            if t:
                state_cpy[sv_offset] = a[3] 
                state_cpy[sv_offset + 1] = a[4] 
            else:
                state_cpy[sv_offset] = a[3]
        else:
            print('Action was not a write to mem or reg.')
            sys.exit()
    return state_cpy

In [204]:
#If we correctly applied the first instruction this sum should equal zero
(~(apply_write_actions(img,to_apply) == sv_trc[0,0,:])).sum()

0

In [205]:
def strip_reads_act(act):
    #Writes to mem/reg respectively.
    if act.ndim == 1:  
        if act[0]%8 == 2 or act[0]%8 == 4:
            return act
        else:
            return np.array([])
    else:
        return act[((act[:,0]%8) == 2) | ((act[:,0]%8) == 4)]

In [206]:
def two_byte_action_mask(act):
    #Isolates 2 byte actions.
    if act.ndim == 1:
        return (act[0] >> 3) == 2
    else:
        return (act[:,0] >> 3) == 2

In [220]:
def fine_grain_sv_trace(init_st, acts):
    #one more than num actions long to incorporate init state
    #width of initial state. 
    writes = strip_reads_act(acts)
    trc = np.zeros( [ writes.shape[0] + 1, init_st.shape[0] ] )
    trc[0] = init_st
    #Slow proof of concept
    assert len(writes) == len(trc[1:])
    old_st = trc[0]
    #This might be trouble.
    #This doesn't write to trc, but a copy. 
    for i in range(writes.shape[0]):
        trc[i+1] = hack(old_st, writes[i])
    return trc

#     for a,st in zip(writes, trc[1:]):
#         #This is hackery.
#         print("old:", old_st)
#         st = hack(old_st, a)
#         print("new:", st)
#         old_st = st
    
#fine_grain_sv_trace(img, trcs[0])
print(img)
print(fine_grain_sv_trace(img, trcs[0,:,:])[:,:8])

[   0.    0.    0. ...,    5.  137.    5.]
[[   0.    0.    0. ...,    0.    0.    0.]
 [   0.    0.    0. ...,    0.   16.    0.]
 [   0.    0.    0. ...,    0.   16.    0.]
 ..., 
 [   0.    0.    0. ...,  255.    0.    0.]
 [  27.    5.    0. ...,    0.    0.    0.]
 [  28.    5.    0. ...,    0.    0.    0.]]


In [208]:
def hack(state, acts):
    acts = strip_reads_act(acts)
    #If the action was a read, return
    if acts.shape[0] == 0:
        return state
    two_byte_act = two_byte_action_mask(acts)
    #Create mask of 2 byte actions
    #This will probably become a bottleneck
#    assert len(two_byte_act) == len(acts)
    state_cpy = copy.deepcopy(state)
    t=two_byte_act
    a = acts
    #Does the header indicate a mem or reg write?
    if (a[0] % 8) == 4:
        #Is this a 2 byte action? 
        if t:
            #There is only one 2 byte reg, the pc.
            #Think this is correct order.
            state_cpy[0] = a[3]
            state_cpy[1] = a[4]
        else:
            #Increment to account for 2 byte pc.
            state_cpy[a[1]+1] = a[3]
    #Memory write.
    elif (a[0] % 8) == 2:
        #Calculate sv offset. Add 8 for regs.
        sv_offset = a[1] + 256*a[2] + 8
        if t:
            state_cpy[sv_offset] = a[3] 
            state_cpy[sv_offset + 1] = a[4] 
        else:
            state_cpy[sv_offset] = a[3]
    else:
        print('Action was not a write to mem or reg.')
        sys.exit(1)
    return state_cpy